In [ ]:
!pip install fastapi[all] uvicorn python-multipart ngrok diffusers transformers accelerate controlnet_aux torch torchvision

In [1]:
!pip install flask flask_cors python-multipart ngrok diffusers transformers accelerate controlnet_aux torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
# 🧠 Load the ControlNet model
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np
import cv2

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")

canny = CannyDetector()

/usr/local/lib/python3.11/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
from flask import Flask, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

In [8]:
from flask import Flask, request, send_file
from flask_cors import CORS
from io import BytesIO
from PIL import Image
import numpy as np
import cv2
import torch
import random


# Dummy image generation logic for placeholder
def canny(image):
    return cv2.Canny(image, 100, 200)


@app.route("/generate", methods=["POST"])
def generate():
    # Parse form fields
    prompt = request.form.get("prompt")
    controlStrength = float(request.form.get("controlStrength"))
    controlGuidanceEnd = float(request.form.get("controlGuidanceEnd"))  # If unused, you can ignore
    seed = int(request.form.get("seed"))
    randomizeSeed = request.form.get("randomizeSeed") == "true"

    if randomizeSeed:
        seed = random.randint(0, 100000)
    generator = torch.manual_seed(seed)

    # Load control image
    control_image_file = request.files.get("controlImage")
    if not control_image_file:
        return {"error": "Missing controlImage file"}, 400

    image_bytes = control_image_file.read()
    image = Image.open(BytesIO(image_bytes)).convert("RGB")
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    canny_image = canny(image)
    canny_image = Image.fromarray(canny_image)

    # Optional referenceImage
    reference_image_file = request.files.get("referenceImage")
    if reference_image_file:
        ref_image_bytes = reference_image_file.read()
        # You can process reference image if needed
        # For example:
        # ref_image = Image.open(BytesIO(ref_image_bytes)).convert("RGB")
        # Use it in your pipeline if needed
        pass

    # Run inference
    output = pipe(
        prompt=prompt,
        negative_prompt="blurry, distorted, low-quality, nsfw",
        image=canny_image,
        num_inference_steps=30,
        guidance_scale=8.0,
        controlnet_conditioning_scale=controlStrength,
        generator=generator
    ).images[0]

    # Return image
    buf = BytesIO()
    output.save(buf, format="PNG")
    buf.seek(0)

    return send_file(buf, mimetype="image/png")


In [14]:
!ngrok config add-authtoken 2wdETNbxpdF42zG0ceqkOsN8V6Y_3XmTUhcfzQ8NoYfWRKkfe

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
!pip install pyngrok

In [ ]:
# 🌐 Ngrok tunnel + Uvicorn launch
from pyngrok import ngrok

# Open tunnel
public_url = ngrok.connect(8000)
print("✨ Public URL:", public_url)

app.run(host='0.0.0.0', port = 8000, debug=False)


✨ Public URL: NgrokTunnel: "https://cff6-34-16-247-114.ngrok-free.app" -> "http://localhost:8000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
Token indices sequence length is longer than the specified maximum sequence length for this model (106 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['harsh shadows . the image should look like a passport or id photo , front - facing with no expression , sharp focus , and photorealistic details .']


  0%|          | 0/30 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [05/May/2025 17:25:08] "POST /generate HTTP/1.1" 200 -
